In [1]:
import os
from dotenv import load_dotenv

import pandas as pd
from sqlalchemy import create_engine, inspect

In [2]:
load_dotenv()

True

In [3]:
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')

In [4]:
DATABASE_URI = f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}"

In [5]:
engine = create_engine(DATABASE_URI)

In [6]:
connection = engine.connect()

In [7]:
def select(sql):
    return pd.read_sql(sql, connection)

In [8]:
inspect(engine).get_table_names()

['active_clients', 'cards', 'categories', 'offers', 'orders']

# Задача: 

**Orders** содержит данные о заказах клиентов, **Categories** - дерево категорий на сайте.


В
вычислить заказы категории “Аксессуары для умных часов и фитнес-браслетов”, происходящие в течение недели после дня заказа категории “Смарт-час  и фит н с-
браслеты”. 
Из этих заказов для каждой подкатегории (Cat4) аксессуаров вывести по топ-5 товаров с наибольшим кол-вом проданных штук.

Итоговый формат может содержать столбцы Cat4, SKU, Qty.

#### Решение без ограничения на топ-5 товаров

In [9]:
sql = '''

    WITH cte as (
                SELECT o.ClientID,
                       c.Cat3,
                       c.Cat4,
                       o.SKU,
                       o.Date
                FROM orders o 
                LEFT JOIN categories c
                ON o.Cat4ID = c.Cat4ID
                WHERE c.Cat3ID = 4583213310474409984
                )
                
    SELECT c.Cat4,
           o.SKU,
           SUM(o.Qty) as Qty
    FROM orders o 
    LEFT JOIN categories c ON o.Cat4ID = c.Cat4ID
    LEFT JOIN cte ON o.ClientID = cte.ClientID 
                 AND DATEDIFF(DAY(o.Date), DAY(cte.Date)) BETWEEN 0 AND 7
    WHERE c.Cat3ID = 5368148236671620096
    GROUP BY c.Cat4, o.SKU
    ORDER BY c.Cat4, Qty DESC
    
      '''

select(sql)

,Cat4,SKU,Qty
0,Зарядные устройства для фитнес-браслетов,291658525,2.0
1,Зарядные устройства для фитнес-браслетов,652194567,2.0
2,Зарядные устройства для фитнес-браслетов,625537438,1.0
3,Зарядные устройства для фитнес-браслетов,597348762,1.0
4,Защитные стекла и пленки для смарт-часов,322418543,3.0
...,...,...,...
110,Ремешки для фитнес-браслетов,636948298,1.0
111,Ремешки для фитнес-браслетов,438590735,1.0
112,Ремешки для фитнес-браслетов,344056802,1.0
113,Ремешки для фитнес-браслетов,634459613,1.0


#### Решение c ограничением на топ-5 товаров по DENSE_RANK() по количеству. Возможно, это получится сделать более красиво, но в целом концепция решения та же, что и без топа, просто всё выброшено в CTE.

In [10]:
sql = '''

    WITH cte AS (
    SELECT o.ClientID,
           c.Cat3,
           c.Cat4,
           o.SKU,
           o.Date
    FROM orders o 
    LEFT JOIN categories c
    ON o.Cat4ID = c.Cat4ID
    WHERE c.Cat3ID = 4583213310474409984
    ),
    
    aggregated_data AS (
        SELECT c.Cat4,
               o.SKU,
               SUM(o.Qty) AS Qty
        FROM orders o 
        LEFT JOIN categories c ON o.Cat4ID = c.Cat4ID
        LEFT JOIN cte ON o.ClientID = cte.ClientID 
                     AND DATEDIFF(DAY(o.Date), DAY(cte.Date)) BETWEEN 0 AND 7
        WHERE c.Cat3ID = 5368148236671620096
        GROUP BY c.Cat4, o.SKU
    ),
    
    ranked_data AS (
        SELECT *,
               DENSE_RANK() OVER (PARTITION BY Cat4 ORDER BY Qty DESC) AS rnk
        FROM aggregated_data
    )
    
    SELECT Cat4,
           SKU,
           Qty
    FROM ranked_data
    WHERE rnk <= 5
    ORDER BY Cat4, Qty DESC;

      '''

select(sql)

,Cat4,SKU,Qty
0,Зарядные устройства для фитнес-браслетов,291658525,2.0
1,Зарядные устройства для фитнес-браслетов,652194567,2.0
2,Зарядные устройства для фитнес-браслетов,625537438,1.0
3,Зарядные устройства для фитнес-браслетов,597348762,1.0
4,Защитные стекла и пленки для смарт-часов,322418543,3.0
...,...,...,...
68,Ремешки для фитнес-браслетов,636948097,7.0
69,Ремешки для фитнес-браслетов,246970208,5.0
70,Ремешки для фитнес-браслетов,634457307,4.0
71,Ремешки для фитнес-браслетов,634457282,4.0


In [11]:
connection.close()